# Negative Binomial Hurdle Model Test
This shows that stan and statsmodels have close results on a simulation dataset. This cross validates stan and statsmodels negative binomial hurdle model implementation. However, on some sample data, statsmodels cannot return log likelihood indicating numerical stability issues.

In [1]:
import sys
sys.path.append("..")
from models.nbh import NBH
sys.path.remove("..")

import numpy as np

In [2]:
def gen_posnb(mu,theta):
    k=1000
    while k>0:
        v=nbinom.rvs(mu, theta)
        if v>0:
            return v
        k=k-1
    return 0

In [3]:
from scipy.stats import uniform, binom, poisson, bernoulli, norm, nbinom
import statsmodels.api as sm

np.random.seed(1)                    # set seed to replicate example
nobs= 20000                          # number of obs in model 

x1 = binom.rvs(1, 0.7, size=nobs)
x2 = norm.rvs(loc=0, scale=1.0, size=nobs)

X = sm.add_constant(x1)
X_infl = X

beta = [1.0, -0.5]
xb = np.dot(X, beta)          # linear predictor

betal = [1.0, -0.3]
xl = np.dot(X, betal)         # linear predictor

exb = np.exp(xb)
exc = 1.0 / (1.0 + np.exp(-xl))

phi = 2

p = bernoulli.rvs(exc)

phy=np.zeros(nobs)
for i in range(nobs):
    if p[i]>0:
        v = gen_posnb(phi, phi/(phi+exb[i]))
        phy[i]=v

In [4]:
mod = NBH(phy,X,exog_infl=X_infl,model_path='../models')

In [5]:
res0=mod.fit(method='stan')[0]
res1=mod.fit(method='statsmodels')[0]

true value of parameters

In [6]:
betal + beta + [phi]

[1.0, -0.3, 1.0, -0.5, 2]

In [7]:
res0

{'params': array([ 1.01770332, -0.31303515,  0.98384227, -0.4842688 ,  0.65985188]),
 'llf_logit': -12356.0791674436,
 'llf_poi': -23919.290889840042,
 'llf': -36275.37005728364,
 'df': 5,
 'aic': 72560.74011456728,
 'cpu_time': 8.004539251327515,
 'model': 'nbh',
 'method': 'stan'}

In [8]:
res1

{'params': array([ 1.01770332, -0.31303515,  0.98384357, -0.48427031,  0.5169239 ]),
 'llf_logit': -12356.0791674436,
 'llf_nb': -23919.290889812364,
 'llf': -36275.37005725596,
 'df': 5,
 'aic': 72560.74011451192,
 'cpu_time': 0.48662805557250977,
 'model': 'nbh',
 'method': 'statsmodels'}

In [9]:
res0['llf']-res1['llf']

-2.7677742764353752e-08

In [10]:
mod.fit_fast()[0]

{'params': array([ 1.01770332, -0.31303515,  0.98384357, -0.48427031,  0.5169239 ]),
 'llf_logit': -12356.0791674436,
 'llf_nb': -23919.290889812364,
 'llf': -36275.37005725596,
 'df': 5,
 'aic': 72560.74011451192,
 'cpu_time': 0.605189323425293,
 'model': 'nbh',
 'method': 'statsmodels'}

statsmodels and stan return the same results.

In [11]:
from scipy.stats import uniform, binom, poisson, bernoulli, norm, nbinom
import statsmodels.api as sm

np.random.seed(1)                    # set seed to replicate example
nobs= 20000                          # number of obs in model 

x1 = binom.rvs(1, 0.7, size=nobs)
x2 = norm.rvs(loc=0, scale=1.0, size=nobs)

X = sm.add_constant(x1)
X_infl = X

beta = [1.0, -0.5]
xb = np.dot(X, beta)          # linear predictor

betal = [1.0, -0.3]
xl = np.dot(X, betal)         # linear predictor

exb = np.exp(xb)
exc = 1.0 / (1.0 + np.exp(-xl))

theta = 0.7

p = bernoulli.rvs(exc)

phy=np.zeros(nobs)
for i in range(nobs):
    if p[i]>0:
        v = gen_posnb(exb[i],theta)
        phy[i]=v

In [12]:
mod = NBH(phy,X,exog_infl=X_infl,model_path='../models')

In [13]:
res0=mod.fit(method='stan')[0]
res1=mod.fit(method='statsmodels')[0]

/Users/taocui/anaconda3/envs/tf1/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:3935: RuntimeWarning: invalid value encountered in log
  a1 * np.log(a1) + y * np.log(mu) -
/Users/taocui/anaconda3/envs/tf1/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:3936: RuntimeWarning: invalid value encountered in log
  (y + a1) * np.log(a2))


In [14]:
res0

{'params': array([ 1.01770332, -0.31303515,  0.09015164, -0.38924294,  0.65576306]),
 'llf_logit': -12356.0791674436,
 'llf_poi': -15574.612655664663,
 'llf': -27930.691823108264,
 'df': 5,
 'aic': 55871.38364621653,
 'cpu_time': 16.279569149017334,
 'model': 'nbh',
 'method': 'stan'}

In [15]:
res1

{'params': array([ 1.01770332, -0.31303515,  0.63552475, -0.17099588, -0.1263608 ]),
 'llf_logit': -12356.0791674436,
 'llf_nb': nan,
 'llf': nan,
 'df': 5,
 'aic': nan,
 'cpu_time': 0.20600104331970215,
 'model': 'nbh',
 'method': 'statsmodels'}

In [16]:
res0['llf']-res1['llf']

nan

In [17]:
mod.fit_fast()[0]

/Users/taocui/anaconda3/envs/tf1/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:3935: RuntimeWarning: invalid value encountered in log
  a1 * np.log(a1) + y * np.log(mu) -
/Users/taocui/anaconda3/envs/tf1/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:3936: RuntimeWarning: invalid value encountered in log
  (y + a1) * np.log(a2))


{'params': array([ 1.01770332, -0.31303515,  0.09015164, -0.38924294,  0.65576306]),
 'llf_logit': -12356.0791674436,
 'llf_poi': -15574.612655664663,
 'llf': -27930.691823108264,
 'df': 5,
 'aic': 55871.38364621653,
 'cpu_time': 16.256245851516724,
 'model': 'nbh',
 'method': 'stan'}

statsmodels incurs numerical errors on a slightly modified simulation dataset.

## Test init

In [18]:
res2=mod.fit(method='stan',start_params=res0['params'])[0]
start_params = res0['params']
start_params[-1]=np.exp(-start_params[-1])
res3=mod.fit(method='statsmodels',start_params=start_params)[0]

In [19]:
res2

{'params': array([ 1.01770332, -0.31303515,  0.09014993, -0.38924401,  0.65576353]),
 'llf_logit': -12356.0791674436,
 'llf_poi': -15574.612655638848,
 'llf': -27930.69182308245,
 'df': 5,
 'aic': 55871.3836461649,
 'cpu_time': 0.3261837959289551,
 'model': 'nbh',
 'method': 'stan'}

In [20]:
res3

{'params': array([ 1.01770332, -0.31303515,  0.09015164, -0.38924294,  0.51904585]),
 'llf_logit': -12356.0791674436,
 'llf_nb': -15574.612655662473,
 'llf': -27930.691823106074,
 'df': 5,
 'aic': 55871.38364621215,
 'cpu_time': 0.1715700626373291,
 'model': 'nbh',
 'method': 'statsmodels'}

In [21]:
res2['llf']-res3['llf']

2.3625034373253584e-08

given the stan results, statsmodels can indeed return results indicating numerical instability in statsmodels.

## Discussion

From the results described above, we find that stan and statsmodels return identical results if statsmodels can return results. In addition, the computing time for stan may be longer than statsmodels for the NBH model. This indicates if we want to apply the NBH model, we can run statsmodels first and if statsmodels does not return results, we can run stan to reduce the total computing time.